In [ ]:
from fredapi import Fred
fred = Fred(api_key="685f7e3646a0a8bdc0deaaea2db1c1f5")
# Monthly real GDP, Treasury yields, unemployment, recession flag:
gdp    = fred.get_series('GDPC1')        # real GDP (monthly interpolated)
spread = fred.get_series('T10Y2Y')       # yield spread
unemp  = fred.get_series('UNRATE')       # unemployment rate
rec    = fred.get_series('USREC')        # 1 if in recession, 0 otherwise


In [7]:
import pandas as pd

# Merge into one DataFrame
df = pd.concat([gdp, spread, unemp, rec], axis=1)
df.columns = ['real_gdp', 'yield_spread', 'unemployment', 'recession']
df.dropna(inplace=True)
df


,real_gdp,yield_spread,unemployment,recession
1976-07-01,6404.895,0.86,7.8,0.0
1976-10-01,6451.177,1.25,7.7,0.0
1977-04-01,6654.466,1.41,7.2,0.0
1977-07-01,6774.457,1.22,6.9,0.0
1979-10-01,7318.535,-0.63,6.0,0.0
...,...,...,...,...
2022-04-01,21919.222,-0.05,3.7,0.0
2022-07-01,22066.784,0.04,3.5,0.0
2024-04-01,23223.906,-0.39,3.9,0.0
2024-07-01,23400.294,-0.29,4.2,0.0


In [9]:
for col in ['yield_spread','unemployment']:
    for lag in [1,3,6]:
        df[f'{col}_lag{lag}'] = df[col].shift(lag)
df.dropna(inplace=True)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(df.drop(columns='recession'))
y = df['recession']
df


,real_gdp,yield_spread,unemployment,recession,yield_spread_lag1,yield_spread_lag3,yield_spread_lag6,unemployment_lag1,unemployment_lag3,unemployment_lag6
1982-07-01,7300.896,-0.20,9.8,1.0,-0.32,-0.90,1.12,9.3,7.2,7.8
1982-10-01,7303.817,0.38,10.4,1.0,-0.20,-0.82,-0.24,9.8,7.9,7.5
1983-07-01,7719.746,0.76,9.4,0.0,0.38,-0.32,-0.05,10.4,9.3,7.2
1984-10-01,8320.199,0.45,7.4,0.0,0.76,-0.20,-0.90,9.4,9.8,7.2
1985-04-01,8474.787,1.21,7.3,0.0,0.45,0.38,-0.82,7.4,10.4,7.9
...,...,...,...,...,...,...,...,...,...,...
2022-04-01,21919.222,-0.05,3.7,0.0,1.21,1.52,0.39,4.5,6.1,14.8
2022-07-01,22066.784,0.04,3.5,0.0,-0.05,1.23,0.52,3.7,5.4,10.2
2024-04-01,23223.906,-0.39,3.9,0.0,0.04,1.21,0.54,3.5,4.5,6.9
2024-07-01,23400.294,-0.29,4.2,0.0,-0.39,-0.05,1.52,3.9,3.7,6.1


In [15]:
# Create forecast target (next period recession)
df['recession_next'] = df['recession'].shift(-1)
df_forecast = df.dropna()  # drop the last row where recession_next is NaN

# Build new features set (drop both recession columns)
X_new = df_forecast.drop(columns=['recession', 'recession_next'])
y_new = df_forecast['recession_next']

# Standardize features
from sklearn.preprocessing import StandardScaler
scaler_new = StandardScaler()
X_new_scaled = scaler_new.fit_transform(X_new)

# Train logistic regression model for forecast
from sklearn.linear_model import LogisticRegression
model_new = LogisticRegression(max_iter=50000)
model_new.fit(X_new_scaled, y_new)

# Forecast recession probability for the next period using the most recent data
X_recent = X_new.tail(1)
X_recent_scaled = scaler_new.transform(X_recent)
forecast_prob = model_new.predict_proba(X_recent_scaled)[0][1]
print("Year:", year, "Forecast recession probability for next period:", forecast_prob)

# Compute and print accuracy on the entire forecast set
from sklearn.metrics import accuracy_score
y_pred = model_new.predict(X_new_scaled)
accuracy = accuracy_score(y_new, y_pred)
print("Forecast Model Accuracy:", accuracy)

Year: 2024 Forecast recession probability for next period: 0.4077955298211745
Forecast Model Accuracy: 0.9080459770114943
